In [1]:
import tkinter as tk
import random
import math
import numpy as np

In [2]:
## The container class

class Bot():
    
    '''This is the class for the bot'''
    
    # This is to intialize the variables 
    def __init__(self,namep):
        self.x = random.randint(100,900)
        self.y = random.randint(100,900)
        self.theta = random.uniform(0.0,2.0*math.pi)
        #self.theta = 0
        self.name = namep
        self.ll = 60 #axle width
        self.vl = 0.0
        self.vr = 0.0
        
    # This is to draw the canvas and all the things on the canvas 
    def draw(self,canvas):
        
         # 4(x,y) coordinates so we can make a rectangle 
        points = [ (self.x + 30*math.sin(self.theta)) - 30*math.sin((math.pi/2.0)-self.theta), \
                   (self.y - 30*math.cos(self.theta)) - 30*math.cos((math.pi/2.0)-self.theta), \
                   (self.x - 30*math.sin(self.theta)) - 30*math.sin((math.pi/2.0)-self.theta), \
                   (self.y + 30*math.cos(self.theta)) - 30*math.cos((math.pi/2.0)-self.theta), \
                   (self.x - 30*math.sin(self.theta)) + 30*math.sin((math.pi/2.0)-self.theta), \
                   (self.y + 30*math.cos(self.theta)) + 30*math.cos((math.pi/2.0)-self.theta), \
                   (self.x + 30*math.sin(self.theta)) + 30*math.sin((math.pi/2.0)-self.theta), \
                   (self.y - 30*math.cos(self.theta)) + 30*math.cos((math.pi/2.0)-self.theta)  \
                ]
        canvas.create_polygon(points, fill="blue", tags=self.name)
        
        ## This for the sensors on the bots
        self.sensorPositions = [ (self.x + 20*math.sin(self.theta)) + 30*math.sin((math.pi/2.0)-self.theta), \
                                 (self.y - 20*math.cos(self.theta)) + 30*math.cos((math.pi/2.0)-self.theta), \
                                 (self.x - 20*math.sin(self.theta)) + 30*math.sin((math.pi/2.0)-self.theta), \
                                 (self.y + 20*math.cos(self.theta)) + 30*math.cos((math.pi/2.0)-self.theta)  \
                            ]
    
        # This is the center point of the bots
        centre1PosX = self.x 
        centre1PosY = self.y
        canvas.create_oval(centre1PosX-8,centre1PosY-8,\
                           centre1PosX+8,centre1PosY+8,\
                           fill="gold",tags=self.name)
        
        # This is the first wheel and its positions
        wheel1PosX = self.x - 30*math.sin(self.theta)
        wheel1PosY = self.y + 30*math.cos(self.theta)
        
        # Draw that wheel on the canvas
        canvas.create_oval(wheel1PosX-3,wheel1PosY-3,\
                                         wheel1PosX+3,wheel1PosY+3,\
                                         fill="red",tags=self.name)
        
        # This is the second wheel and its positions
        wheel2PosX = self.x + 30*math.sin(self.theta)
        wheel2PosY = self.y - 30*math.cos(self.theta)
        
        # Draw that wheel on the canvas
        canvas.create_oval(wheel2PosX-3,wheel2PosY-3,\
                                         wheel2PosX+3,wheel2PosY+3,\
                                         fill="green",tags=self.name)
        
        
        # We have two sensors so this tell the postions of the sensors
        
        # Odd place has the positions for the Y coordinate
        # Even places has the positions for the X coordinate
        
        # First sensors X and Y coordinate
        sensor1PosX = self.sensorPositions[0]
        sensor1PosY = self.sensorPositions[1]
        
        
        # Second sensors X and Y coordinates 
        sensor2PosX = self.sensorPositions[2]
        sensor2PosY = self.sensorPositions[3]
        
        
        ## Creating the sensors on the canvas
        canvas.create_oval(sensor1PosX-3,sensor1PosY-3, \
                           sensor1PosX+3,sensor1PosY+3, \
                           fill="yellow",tags=self.name)
        canvas.create_oval(sensor2PosX-3,sensor2PosY-3, \
                           sensor2PosX+3,sensor2PosY+3, \
                           fill="yellow",tags=self.name)
        
    # cf. Dudek and Jenkin, Computational Principles of Mobile Robotics # Refrence for the code
    def move(self,canvas,dt):
        '''This is the function that is responsible for the moments of the robot'''
        
        # vl refers to the velocity of left wheel and vr for the right wheel
        # R refers to the coeeficent of Rotation
        if self.vl==self.vr:
            # This basically means a striaght line
            # The Force balance out
            R = 0
        else:
            # This is the Rotation
            # If the velocites are different of the wheels then there will be some rotation
            # It depends where the rotation will be depending on the magnitude of the force 
            
            R = (self.ll/2.0)*((self.vr+self.vl)/(self.vl-self.vr))
            
            
        omega = (self.vl-self.vr)/self.ll ## TODO come up with an explantion for this
        ICCx = self.x-R*math.sin(self.theta) #instantaneous centre of curvature in X coordinate
        ICCy = self.y+R*math.cos(self.theta) #instantaneous centre of curvature in Y coordinate
        
        # This matrix has 3 x 3 dimensions 
        ## TODO to comeup with the significance of this matrix
        m = np.matrix( [ [math.cos(omega*dt), -math.sin(omega*dt), 0], \
                        [math.sin(omega*dt), math.cos(omega*dt), 0],  \
                        [0,0,1] ] )
        
        # These two matrix are basicall 3 X 1 dimensions
        v1 = np.matrix([[self.x-ICCx],[self.y-ICCy],[self.theta]])
        v2 = np.matrix([[ICCx],[ICCy],[omega*dt]])
        
        # Think of this value as wTx in machine learning where W is m and v1 is X and the bias is v2 
        newv = np.add(np.dot(m,v1),v2)
        
        # As the result obtained are as a object type we use np.item
        newX = newv.item(0) # 0 contains the new X coordinate
        newY = newv.item(1) # 1 contains the new Y coordinate
        newTheta = newv.item(2) # 2 contains the new Θ coordinate
        
        
        newTheta = newTheta%(2.0*math.pi) #make sure angle doesn't go outside [0.0,2*pi)
        
        # Update the coordinates for the data members 
        self.x = newX
        self.y = newY
        self.theta = newTheta        
        
        ## If the velocity is same then we observe a moment in a straight line
        
        if self.vl==self.vr: # straight line movement
            
            ## TODO to comeup with a reasoning for this
            self.x += self.vr*math.cos(self.theta) #vr wlog
            self.y += self.vr*math.sin(self.theta)
        
        # Removing the last instance from the canvas 
        canvas.delete(self.name)
        
        # Adding the new instance on the canvas
        self.draw(canvas)

    def senseLight(self, registryPassives):
        '''This is a function to set luminous power/Light radius on the canvas'''
        lightL = 0.0
        lightR = 0.0
        
        # Registry Passive means that we are checking for all the lamps that we intialized
        for pp in registryPassives:
            
            # Check whether the object present is an instance of the object we have
            if isinstance(pp,Lamp):
                
                # If it is a instance we get its location
                lx,ly = pp.getLocation()
                
                # We calculate its distance using Euclidean Formula(I guess correct me if I am wrong)
                distanceL = math.sqrt( (lx-self.sensorPositions[0])*(lx-self.sensorPositions[0]) + \
                                       (ly-self.sensorPositions[1])*(ly-self.sensorPositions[1]) )
                distanceR = math.sqrt( (lx-self.sensorPositions[2])*(lx-self.sensorPositions[2]) + \
                                       (ly-self.sensorPositions[3])*(ly-self.sensorPositions[3]) )
                
                # We set the influence or the power of the light
                lightL += 200000/(distanceL*distanceL)
                lightR += 200000/(distanceR*distanceR)
                
        # Return the power of the light        
        return lightL, lightR

    def transferFunction(self, lightL, lightR):
        '''This function is responsible for the moment of the wheels'''
        ## Change the value for a constant moving
        #self.vl = 0.0
        #self.vr = 0.0
        # Change the value for a variable moment 
        # What I think is that when it gets close to the light the moment will change
        # So the moment should be dependent on the lightL and lightR value
        self.vl = .5 * lightL  
        self.vr = .5 * lightR
        # This will cause a circular moment because the velocity on the wheels are different
        # Tip play with these variables to understand it more intutively
        

In [3]:
class Lamp():
    
    '''Container class for the lamp object'''
    
    def __init__(self,namep):
        '''Intialize Function/ Constructor'''
        self.centreX = random.randint(100,900)
        self.centreY = random.randint(100,900)
        self.name = namep
        
    def draw(self,canvas):
        '''Drawing the lamps on the instance of canvas'''
        body = canvas.create_oval(self.centreX-10,self.centreY-10, \
                                  self.centreX+10,self.centreY+10, \
                                  fill="yellow",tags=self.name)

    def getLocation(self):
        '''Returning the location of X and Y corrdinate'''
        return self.centreX, self.centreY

In [4]:
def initialise(window):
    '''Intialize the window on the canvas'''
    window.resizable(False,False)
    canvas = tk.Canvas(window,width=1000,height=1000)
    canvas.pack()
    return canvas

In [5]:
def register(canvas):
    '''Register to store all the ojects on canvas'''
    
    # Think of it as a hashmap 
    
    registryActives = [] # Container to store Bot objects
    registryPassives = [] # Container to store Lamp objects
    noOfBots = 4 # Increase the number of bots to see chaos
    noOfLights = 5 # Increase the number of lamps to see chaos
    
    # Interate over the bots frequency and appending them to the list 
    for i in range(0,noOfBots):
        bot = Bot("Bot"+str(i))
        registryActives.append(bot)
        bot.draw(canvas)
    
    # Iterating over the lamp frequency and appending them to the list
    for i in range(0,noOfLights):
        lamp = Lamp("Lamp"+str(i))
        registryPassives.append(lamp)
        lamp.draw(canvas)
    # Returning the list of bots and lamp objects
    return registryActives, registryPassives

In [6]:
def moveIt(canvas,registryActives,registryPassives):
    # Now iterating over the objects in bots list
    for rr in registryActives:
        
        # We callculate the intensity or the range of light for the specific lamp
        lightIntensityL, lightIntensityR = rr.senseLight(registryPassives)
        
        # We transfer the intenstiy on Bot (Check methods of the bot class)
        rr.transferFunction(lightIntensityL, lightIntensityR)
        
        # Moving the bot 
        rr.move(canvas,1.0)
    # Recursive call ? after every 50 milliseconds (I may be wrong here)
    canvas.after(50,moveIt,canvas,registryActives,registryPassives)

In [7]:
def main():
    
    # Making the window to show canvas
    window = tk.Tk()
    
    # Intiallize the canvas on the window
    canvas = initialise(window)
    
    # Making object containers for the bots and the lamps
    registryActives, registryPassives = register(canvas)
    
    # Moving them interatively
    moveIt(canvas,registryActives,registryPassives)
    window.mainloop()

In [8]:
# Calling the main function
main()